# Data Processing Steps

### Read in the individual csv files, concatenate each year's arrest records together into consolidated data frames

In [1]:
import pandas as pd
import numpy as np

In [2]:
arrestee_2020 = pd.read_csv("2020_NIBRS_ARRESTEE.csv")
arrestee_2019 = pd.read_csv("2019_NIBRS_ARRESTEE.csv")
arrestee_2018 = pd.read_csv("2018_NIBRS_ARRESTEE.csv")
arrestee_2017 = pd.read_csv("2017_NIBRS_ARRESTEE.csv")
arrestee_frames = [arrestee_2020,arrestee_2019,arrestee_2018,arrestee_2017]
arrestee_all = pd.concat(arrestee_frames)
#arrestee_all.head()

incident_2020 = pd.read_csv("2020_NIBRS_incident.csv")
incident_2019 = pd.read_csv("2019_NIBRS_incident.csv")
incident_2018 = pd.read_csv("2018_NIBRS_incident.csv")
incident_2017 = pd.read_csv("2017_NIBRS_incident.csv")
incident_frames = [incident_2020,incident_2019,incident_2018,incident_2017]
incident_all = pd.concat(incident_frames)
#incident_all.head()

offense_2020 = pd.read_csv("2020_NIBRS_OFFENSE.csv")
offense_2019 = pd.read_csv("2019_NIBRS_OFFENSE.csv")
offense_2018 = pd.read_csv("2018_NIBRS_OFFENSE.csv")
offense_2017 = pd.read_csv("2017_NIBRS_OFFENSE.csv")
offense_frames = [offense_2020,offense_2019,offense_2018,offense_2017]
offense_all = pd.concat(offense_frames)
#offense_all.head()

using_2020 = pd.read_csv("2020_NIBRS_SUSPECT_USING.csv")
using_2019 = pd.read_csv("2019_NIBRS_SUSPECT_USING.csv")
using_2018 = pd.read_csv("2018_NIBRS_SUSPECT_USING.csv")
using_2017 = pd.read_csv("2017_NIBRS_SUSPECT_USING.csv")
using_frames = [using_2020,using_2019,using_2018,using_2017]
using_all = pd.concat(using_frames)

agency_2020 = pd.read_csv("2020_agencies.csv")

offense_type = pd.read_csv('NIBRS_OFFENSE_TYPE.csv')


C:\Users\becky\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Keep only the relevant columns for each new dataframe. Adjust column names when necessary. Remove any rows where there are NaN values (we will only be utilizing data records that have been completely populated for the purpose of our data analysis) 

#### Arrestee Dataframe

In [3]:
arrestee_all = arrestee_all[['OFFENSE_TYPE_ID','AGE_NUM','SEX_CODE','RACE_ID','RESIDENT_CODE',
                             'ARRESTEE_ID','INCIDENT_ID']]
arrestee_all = arrestee_all.rename(columns={'AGE_NUM': 'ARR_AGE_NUM', 
                                          'SEX_CODE': 'ARR_SEX_CODE','RACE_ID':'ARR_RACE_ID',
                                          'RESIDENT_CODE':'ARR_RESIDENT_CODE'})
arrestee_all.dropna(inplace=True)

arrestee_all.head()

,OFFENSE_TYPE_ID,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,ARRESTEE_ID,INCIDENT_ID
0,27,86.0,M,1,R,41972966,135853809
1,51,33.0,M,1,R,41975901,135853845
2,51,28.0,M,1,R,41976088,135857893
3,27,30.0,M,1,R,41984629,135878943
6,16,19.0,F,2,N,41753775,135155097


#### Incident Dataframe

In [4]:
incident_all = incident_all[['INCIDENT_DATE','INCIDENT_HOUR','AGENCY_ID','INCIDENT_ID']]

incident_all.dropna(inplace=True)

incident_all.head()

,INCIDENT_DATE,INCIDENT_HOUR,AGENCY_ID,INCIDENT_ID
0,08-DEC-20,11.0,29534,133835511
1,14-DEC-20,16.0,29534,133835521
2,07-DEC-20,16.0,29534,133831060
3,03-MAR-20,16.0,29534,123548684
4,04-MAR-20,15.0,29534,123548754


#### Offense Dataframe

In [5]:
offense_all = offense_all[['INCIDENT_ID','OFFENSE_ID','LOCATION_ID']]

offense_all.dropna(inplace=True)

offense_all.head()

,INCIDENT_ID,OFFENSE_ID,LOCATION_ID
0,124796257,151187996,20
1,124796312,151183350,20
2,124796411,151188163,20
3,124796411,151188162,20
4,124796494,151183579,20


#### Using Dataframe  (suspect was under the influence) 

In [6]:
using_all = using_all[['SUSPECT_USING_ID','OFFENSE_ID']]

using_all.dropna(inplace=True)

using_all.head()

,SUSPECT_USING_ID,OFFENSE_ID
0,4,162613628
1,4,162609062
2,4,162609076
3,4,162609075
4,4,162613686


In [7]:
# convert the suspect using id to a binary 1= Under the Influence  0= Not Using

using_all['SUSPECT_USING_ID'].replace({1:1, 3:1, 2:0,4:0}, inplace=True)

using_all.head()

,SUSPECT_USING_ID,OFFENSE_ID
0,0,162613628
1,0,162609062
2,0,162609076
3,0,162609075
4,0,162613686


In [8]:
using_all['SUSPECT_USING_ID'].value_counts()

0    2847148
1     297599
Name: SUSPECT_USING_ID, dtype: int64

### Merge arrestee, incident, offense and using dataframes together. Remove any data records with NaN values.

In [9]:
a = arrestee_all
i = incident_all
o = offense_all
u = using_all

ai_df = pd.merge(a,i,on='INCIDENT_ID')
ai_df.dropna(inplace=True)

aio_df = pd.merge(ai_df,o,on='INCIDENT_ID')
aio_df.dropna(inplace=True)

aiou_df = pd.merge(aio_df,u,on='OFFENSE_ID')
aiou_df.dropna(inplace=True)

aiou_df.head()

,OFFENSE_TYPE_ID,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,ARRESTEE_ID,INCIDENT_ID,INCIDENT_DATE,INCIDENT_HOUR,AGENCY_ID,OFFENSE_ID,LOCATION_ID,SUSPECT_USING_ID
0,27,86.0,M,1,R,41972966,135853809,17-NOV-20,15.0,19409,163618082,20,0
1,51,33.0,M,1,R,41975901,135853845,24-DEC-20,1.0,19409,163621876,20,1
2,51,28.0,M,1,R,41976088,135857893,10-SEP-20,21.0,18796,163622470,20,0
3,27,30.0,M,1,R,41984629,135878943,15-DEC-20,18.0,19711,163644280,20,1
4,16,19.0,F,2,N,41753775,135155097,20-DEC-20,17.0,19870,162738277,18,0


In [10]:
aiou_df.count()

OFFENSE_TYPE_ID      702030
ARR_AGE_NUM          702030
ARR_SEX_CODE         702030
ARR_RACE_ID          702030
ARR_RESIDENT_CODE    702030
ARRESTEE_ID          702030
INCIDENT_ID          702030
INCIDENT_DATE        702030
INCIDENT_HOUR        702030
AGENCY_ID            702030
OFFENSE_ID           702030
LOCATION_ID          702030
SUSPECT_USING_ID     702030
dtype: int64

### Prepare the Agencies Dataframe to merge with the aiou_df

In [11]:
agency_2020 = agency_2020[['AGENCY_ID',
        'AGENCY_TYPE_NAME', 'POPULATION', 'SUBURBAN_AREA_FLAG',
        'PED.MALE_OFFICER+PED.MALE_CIVILIAN',
        'PED.FEMALE_CIVILIAN+PED.FEMALE_OFFICER','COUNTY_NAME']]

agency_2020['SUBURBAN_AREA_FLAG'].replace({'N':1, 'Y':2}, inplace=True)

agency_2020 = agency_2020.rename(columns={'PED.MALE_OFFICER+PED.MALE_CIVILIAN': 'MALE_EMPLOYEES', 
                        'PED.FEMALE_CIVILIAN+PED.FEMALE_OFFICER':'FEMALE_EMPLOYEES'})

agency_2020['TOTAL_EMPLOYEES'] = agency_2020['MALE_EMPLOYEES']+agency_2020['FEMALE_EMPLOYEES']

agency_2020=agency_2020[['AGENCY_ID','AGENCY_TYPE_NAME', 'POPULATION', 'SUBURBAN_AREA_FLAG','TOTAL_EMPLOYEES','COUNTY_NAME']]

agency_2020.head()

,AGENCY_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,COUNTY_NAME
0,18753,City,17625,1,43.0,ANDERSON
1,18754,City,1164,1,5.0,HENDERSON; ANDERSON
2,18757,City,14487,1,19.0,ANDREWS
3,18759,City,5182,1,17.0,ANGELINA
4,18760,City,2110,1,6.0,ANGELINA


### Merge the Agencies Dataframe with the aiou_df

In [12]:
ag = agency_2020

aiouag_df = pd.merge(aiou_df,ag,on='AGENCY_ID')

aiouag_df.dropna(inplace=True)

aiouag_df.head()

# It is worth noting that it may look like some records in the dataframe are duplicates; however,
# an arrestee can be charged with multiple offenses during a single incident. We will keep each offense
# for the purpose of our analysis

,OFFENSE_TYPE_ID,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,ARRESTEE_ID,INCIDENT_ID,INCIDENT_DATE,INCIDENT_HOUR,AGENCY_ID,OFFENSE_ID,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,COUNTY_NAME
0,27,86.0,M,1,R,41972966,135853809,17-NOV-20,15.0,19409,163618082,20,0,County,77723,2,212.0,KAUFMAN
1,51,33.0,M,1,R,41975901,135853845,24-DEC-20,1.0,19409,163621876,20,1,County,77723,2,212.0,KAUFMAN
2,48,25.0,M,1,R,40508807,130983780,25-AUG-20,0.0,19409,157975082,20,1,County,77723,2,212.0,KAUFMAN
3,16,38.0,M,1,N,40508811,130980608,01-SEP-20,22.0,19409,157975088,13,1,County,77723,2,212.0,KAUFMAN
4,16,38.0,M,1,N,40508811,130980608,01-SEP-20,22.0,19409,157975089,13,1,County,77723,2,212.0,KAUFMAN


In [13]:
aiouag_df.count()

OFFENSE_TYPE_ID       683410
ARR_AGE_NUM           683410
ARR_SEX_CODE          683410
ARR_RACE_ID           683410
ARR_RESIDENT_CODE     683410
ARRESTEE_ID           683410
INCIDENT_ID           683410
INCIDENT_DATE         683410
INCIDENT_HOUR         683410
AGENCY_ID             683410
OFFENSE_ID            683410
LOCATION_ID           683410
SUSPECT_USING_ID      683410
AGENCY_TYPE_NAME      683410
POPULATION            683410
SUBURBAN_AREA_FLAG    683410
TOTAL_EMPLOYEES       683410
COUNTY_NAME           683410
dtype: int64

### Prepare and merge the Offense Type Dataframe with the aiouag_df. The Offense Type Dataframe will provide our dependent variable: 'Crime Against'

In [14]:
offense_type = offense_type[['OFFENSE_TYPE_ID','CRIME_AGAINST']]

ot = offense_type

aiouagot_df = pd.merge(aiouag_df,ot,on="OFFENSE_TYPE_ID")

aiouagot_df.head()

,OFFENSE_TYPE_ID,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,ARRESTEE_ID,INCIDENT_ID,INCIDENT_DATE,INCIDENT_HOUR,AGENCY_ID,OFFENSE_ID,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,COUNTY_NAME,CRIME_AGAINST
0,27,86.0,M,1,R,41972966,135853809,17-NOV-20,15.0,19409,163618082,20,0,County,77723,2,212.0,KAUFMAN,Person
1,27,24.0,M,1,R,41975866,135857255,08-OCT-20,14.0,19409,163621794,20,0,County,77723,2,212.0,KAUFMAN,Person
2,27,36.0,M,1,R,41972964,135857273,17-NOV-20,22.0,19409,163621812,20,0,County,77723,2,212.0,KAUFMAN,Person
3,27,47.0,F,2,R,38581711,124762269,06-MAY-20,14.0,19409,151145579,13,0,County,77723,2,212.0,KAUFMAN,Person
4,27,19.0,M,2,R,38581712,124762269,06-MAY-20,14.0,19409,151145579,13,0,County,77723,2,212.0,KAUFMAN,Person


### The Offense Type column must now be removed because it has a direct hierarchal relationship with the Crime Against column (Crime Against is the parent category).

In [15]:
aiouagot_df = aiouagot_df.drop('OFFENSE_TYPE_ID', 1)

aiouagot_df.head()

,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,ARRESTEE_ID,INCIDENT_ID,INCIDENT_DATE,INCIDENT_HOUR,AGENCY_ID,OFFENSE_ID,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,COUNTY_NAME,CRIME_AGAINST
0,86.0,M,1,R,41972966,135853809,17-NOV-20,15.0,19409,163618082,20,0,County,77723,2,212.0,KAUFMAN,Person
1,24.0,M,1,R,41975866,135857255,08-OCT-20,14.0,19409,163621794,20,0,County,77723,2,212.0,KAUFMAN,Person
2,36.0,M,1,R,41972964,135857273,17-NOV-20,22.0,19409,163621812,20,0,County,77723,2,212.0,KAUFMAN,Person
3,47.0,F,2,R,38581711,124762269,06-MAY-20,14.0,19409,151145579,13,0,County,77723,2,212.0,KAUFMAN,Person
4,19.0,M,2,R,38581712,124762269,06-MAY-20,14.0,19409,151145579,13,0,County,77723,2,212.0,KAUFMAN,Person


### Convert the Incident Date to a DateTime format and generate an Incident Month column in aiouagot_df

In [16]:
aiouagot_df['INCIDENT_DATE'] = pd.to_datetime(aiouagot_df['INCIDENT_DATE'])

aiouagot_df['INCIDENT_MONTH'] = pd.DatetimeIndex(aiouagot_df['INCIDENT_DATE']).month

aiouagot_df['INCIDENT_YEAR'] = pd.DatetimeIndex(aiouagot_df['INCIDENT_DATE']).year

aiouagot_df.head()

,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,ARRESTEE_ID,INCIDENT_ID,INCIDENT_DATE,INCIDENT_HOUR,AGENCY_ID,OFFENSE_ID,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,COUNTY_NAME,CRIME_AGAINST,INCIDENT_MONTH,INCIDENT_YEAR
0,86.0,M,1,R,41972966,135853809,2020-11-17,15.0,19409,163618082,20,0,County,77723,2,212.0,KAUFMAN,Person,11,2020
1,24.0,M,1,R,41975866,135857255,2020-10-08,14.0,19409,163621794,20,0,County,77723,2,212.0,KAUFMAN,Person,10,2020
2,36.0,M,1,R,41972964,135857273,2020-11-17,22.0,19409,163621812,20,0,County,77723,2,212.0,KAUFMAN,Person,11,2020
3,47.0,F,2,R,38581711,124762269,2020-05-06,14.0,19409,151145579,13,0,County,77723,2,212.0,KAUFMAN,Person,5,2020
4,19.0,M,2,R,38581712,124762269,2020-05-06,14.0,19409,151145579,13,0,County,77723,2,212.0,KAUFMAN,Person,5,2020


### Drop Unnecessary ID fields now that all the data has been merged

In [17]:
aiouagot_df.columns

Index(['ARR_AGE_NUM', 'ARR_SEX_CODE', 'ARR_RACE_ID', 'ARR_RESIDENT_CODE',
       'ARRESTEE_ID', 'INCIDENT_ID', 'INCIDENT_DATE', 'INCIDENT_HOUR',
       'AGENCY_ID', 'OFFENSE_ID', 'LOCATION_ID', 'SUSPECT_USING_ID',
       'AGENCY_TYPE_NAME', 'POPULATION', 'SUBURBAN_AREA_FLAG',
       'TOTAL_EMPLOYEES', 'COUNTY_NAME', 'CRIME_AGAINST', 'INCIDENT_MONTH',
       'INCIDENT_YEAR'],
      dtype='object')

In [18]:
aiouagot_df = aiouagot_df[['ARR_AGE_NUM', 'ARR_SEX_CODE', 'ARR_RACE_ID',
       'ARR_RESIDENT_CODE','INCIDENT_HOUR','INCIDENT_MONTH','INCIDENT_YEAR',
        'LOCATION_ID','SUSPECT_USING_ID', 'AGENCY_TYPE_NAME', 'POPULATION',
       'SUBURBAN_AREA_FLAG', 'TOTAL_EMPLOYEES', 'CRIME_AGAINST','COUNTY_NAME']]

aiouagot_df.head()

,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,INCIDENT_HOUR,INCIDENT_MONTH,INCIDENT_YEAR,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,CRIME_AGAINST,COUNTY_NAME
0,86.0,M,1,R,15.0,11,2020,20,0,County,77723,2,212.0,Person,KAUFMAN
1,24.0,M,1,R,14.0,10,2020,20,0,County,77723,2,212.0,Person,KAUFMAN
2,36.0,M,1,R,22.0,11,2020,20,0,County,77723,2,212.0,Person,KAUFMAN
3,47.0,F,2,R,14.0,5,2020,13,0,County,77723,2,212.0,Person,KAUFMAN
4,19.0,M,2,R,14.0,5,2020,13,0,County,77723,2,212.0,Person,KAUFMAN


### Review all categorical fields to remove any remaining data records with "unknowns" from the dataframe as theses were likely missed during the NaN removal steps.

In [19]:
print(aiouagot_df['ARR_SEX_CODE'].value_counts())
print(aiouagot_df['ARR_RACE_ID'].value_counts())
print(aiouagot_df['ARR_RESIDENT_CODE'].value_counts())
print(aiouagot_df['CRIME_AGAINST'].value_counts())

M    502478
F    180932
Name: ARR_SEX_CODE, dtype: int64
1    446291
2    219163
4      7766
8      6113
0      3243
3       834
Name: ARR_RACE_ID, dtype: int64
R    443411
N    197165
U     42834
Name: ARR_RESIDENT_CODE, dtype: int64
Society        315769
Person         189699
Property       177910
Not a Crime        32
Name: CRIME_AGAINST, dtype: int64


In [20]:
# ARR_RACE_ID: 0 = 'Uknown'
# ARR_RESIDENT_CODE: U = 'Unknown'

aiouagot_df.drop(aiouagot_df[aiouagot_df['ARR_RACE_ID'] == 0].index, inplace = True)

aiouagot_df.drop(aiouagot_df[aiouagot_df['ARR_RESIDENT_CODE'] == 'U'].index, inplace = True)

aiouagot_df.drop(aiouagot_df[aiouagot_df['CRIME_AGAINST'] == 'Not a Crime'].index, inplace = True)

aiouagot_df.head()

,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,INCIDENT_HOUR,INCIDENT_MONTH,INCIDENT_YEAR,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,CRIME_AGAINST,COUNTY_NAME
0,86.0,M,1,R,15.0,11,2020,20,0,County,77723,2,212.0,Person,KAUFMAN
1,24.0,M,1,R,14.0,10,2020,20,0,County,77723,2,212.0,Person,KAUFMAN
2,36.0,M,1,R,22.0,11,2020,20,0,County,77723,2,212.0,Person,KAUFMAN
3,47.0,F,2,R,14.0,5,2020,13,0,County,77723,2,212.0,Person,KAUFMAN
4,19.0,M,2,R,14.0,5,2020,13,0,County,77723,2,212.0,Person,KAUFMAN


In [21]:
aiouagot_df.count()

ARR_AGE_NUM           637608
ARR_SEX_CODE          637608
ARR_RACE_ID           637608
ARR_RESIDENT_CODE     637608
INCIDENT_HOUR         637608
INCIDENT_MONTH        637608
INCIDENT_YEAR         637608
LOCATION_ID           637608
SUSPECT_USING_ID      637608
AGENCY_TYPE_NAME      637608
POPULATION            637608
SUBURBAN_AREA_FLAG    637608
TOTAL_EMPLOYEES       637608
CRIME_AGAINST         637608
COUNTY_NAME           637608
dtype: int64

### Convert string categorical columns to numerical for future machine learning models. Remove any remaining columns that will not be used for analysis

In [30]:
ml_df = aiouagot_df

ml_df['ARR_SEX_CODE'].replace({'M':1, 'F':0}, inplace=True)
ml_df['ARR_RESIDENT_CODE'].replace({'R':1, 'N':0}, inplace=True)
ml_df['CRIME_AGAINST'].replace({'Person':1, 'Property':2, 'Society': 3}, inplace=True)
ml_df['AGENCY_TYPE_NAME'].replace({'City':1, 'County':2, 'Other':0,'University or College':0, 'Tribal':0}, inplace=True)

ml_df = ml_df[['ARR_AGE_NUM', 'ARR_SEX_CODE', 'ARR_RACE_ID', 'ARR_RESIDENT_CODE',
       'INCIDENT_HOUR', 'INCIDENT_MONTH','LOCATION_ID',
       'SUSPECT_USING_ID', 'AGENCY_TYPE_NAME', 'POPULATION',
       'SUBURBAN_AREA_FLAG', 'TOTAL_EMPLOYEES', 'CRIME_AGAINST']]
ml_df.head(25)

,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,INCIDENT_HOUR,INCIDENT_MONTH,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,CRIME_AGAINST
0,86.0,1,1,1,15.0,11,20,0,2,77723,2,212.0,1
1,24.0,1,1,1,14.0,10,20,0,2,77723,2,212.0,1
2,36.0,1,1,1,22.0,11,20,0,2,77723,2,212.0,1
3,47.0,0,2,1,14.0,5,13,0,2,77723,2,212.0,1
4,19.0,1,2,1,14.0,5,13,0,2,77723,2,212.0,1
5,37.0,1,1,0,15.0,9,20,0,2,77723,2,212.0,1
6,36.0,1,1,1,1.0,2,20,1,2,77723,2,212.0,1
7,18.0,1,2,0,8.0,3,20,0,2,77723,2,212.0,1
8,17.0,1,2,1,14.0,7,15,0,2,77723,2,212.0,1
9,52.0,1,2,1,20.0,4,20,0,2,77723,2,212.0,1


### Export ml_df to a csv to publish to AWS and share with team members conducting the machine learning analysis

In [23]:
#ml_df.to_csv('ml_tx_arrests.csv')

### Create additional csv’s for Tableau Dashboard visualizations. Group by county, year and age group to generate additional information that can be used for geographical representation of the arrest data.

#### Create a dataframe that provides the total offenses resulting in arrest per county (2017-2020). Find the average offense total per county.

In [24]:
# Convert the COUNTY_NAME values in the dataframe from all capital letters to title case
# SOURCE: https://datatofish.com/uppercase-pandas-dataframe/

aiouagot_df['COUNTY_NAME'] = aiouagot_df['COUNTY_NAME'].str.title()

tb_ar_df = aiouagot_df

tb_ar_df['TOT_OFFENSES'] = 1

tb_ar_df = tb_ar_df[['COUNTY_NAME','INCIDENT_YEAR','TOT_OFFENSES']]

tb_ar_df = tb_ar_df.groupby(['COUNTY_NAME','INCIDENT_YEAR']).sum()

tb_ar_df.head(10)

TOT_OFFENSES
COUNTY_NAME INCIDENT_YEAR              
Anderson    2017                    195
            2018                     56
            2019                     77
            2020                    124
Andrews     2019                    211
            2020                    285
Angelina    2019                      3
Aransas     2019                      9
            2020                    154
Atascosa    2019                    168

In [25]:
tb_ar_avg_df = tb_ar_df

tb_ar_avg_df = tb_ar_avg_df.groupby(['COUNTY_NAME']).mean()

tb_ar_avg_df.head(10)

,TOT_OFFENSES
COUNTY_NAME,
Anderson,113.00
Andrews,248.00
Angelina,3.00
Aransas,81.50
Atascosa,293.50
Austin,105.00
Bandera,71.50
Bastrop,146.50
Baylor,38.25


#### Create a dataframe that provides the total offenses by age ranges resulting in arrest per county (2017-2020). Find the average offense, per age range total per county.

In [26]:
tb_age_df = aiouagot_df

tb_age_df['TOT_OFFENSES'] = 1

tb_age_df['AGE_GROUP'] = ""

#Group arrestee ages into age ranges

tb_age_df.loc[tb_age_df['ARR_AGE_NUM'] < 10, 'AGE_GROUP'] = '< 10'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 10) & (tb_age_df['ARR_AGE_NUM'] <= 19),'AGE_GROUP'] = '10-19'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 20) & (tb_age_df['ARR_AGE_NUM'] <= 29),'AGE_GROUP'] = '20-29'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 30) & (tb_age_df['ARR_AGE_NUM'] <= 39),'AGE_GROUP'] = '30-39'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 40) & (tb_age_df['ARR_AGE_NUM'] <= 49),'AGE_GROUP'] = '40-49'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 50) & (tb_age_df['ARR_AGE_NUM'] <= 59),'AGE_GROUP'] = '50-59'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 60) & (tb_age_df['ARR_AGE_NUM'] <= 69),'AGE_GROUP'] = '60-69'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 70) & (tb_age_df['ARR_AGE_NUM'] <= 79),'AGE_GROUP'] = '70-79'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 80) & (tb_age_df['ARR_AGE_NUM'] <= 89),'AGE_GROUP'] = '80-89'
tb_age_df.loc[(tb_age_df['ARR_AGE_NUM'] >= 90) & (tb_age_df['ARR_AGE_NUM'] <= 100),'AGE_GROUP'] = '90+'
              
tb_age_df.head()


,ARR_AGE_NUM,ARR_SEX_CODE,ARR_RACE_ID,ARR_RESIDENT_CODE,INCIDENT_HOUR,INCIDENT_MONTH,INCIDENT_YEAR,LOCATION_ID,SUSPECT_USING_ID,AGENCY_TYPE_NAME,POPULATION,SUBURBAN_AREA_FLAG,TOTAL_EMPLOYEES,CRIME_AGAINST,COUNTY_NAME,TOT_OFFENSES,AGE_GROUP
0,86.0,1,1,1,15.0,11,2020,20,0,2,77723,2,212.0,1,Kaufman,1,80-89
1,24.0,1,1,1,14.0,10,2020,20,0,2,77723,2,212.0,1,Kaufman,1,20-29
2,36.0,1,1,1,22.0,11,2020,20,0,2,77723,2,212.0,1,Kaufman,1,30-39
3,47.0,0,2,1,14.0,5,2020,13,0,2,77723,2,212.0,1,Kaufman,1,40-49
4,19.0,1,2,1,14.0,5,2020,13,0,2,77723,2,212.0,1,Kaufman,1,10-19


In [27]:
tb_age_group_df = tb_age_df[['COUNTY_NAME','INCIDENT_YEAR','AGE_GROUP','TOT_OFFENSES']]

tb_age_group_df = tb_age_group_df.groupby(['COUNTY_NAME','INCIDENT_YEAR','AGE_GROUP']).sum()

tb_age_group_df.head()

TOT_OFFENSES
COUNTY_NAME INCIDENT_YEAR AGE_GROUP              
Anderson    2017          10-19                38
                          20-29                71
                          30-39                46
                          40-49                17
                          50-59                19

In [28]:
tb_age_avg_df = tb_age_group_df

tb_age_avg_df = tb_age_avg_df.groupby(['COUNTY_NAME','AGE_GROUP']).mean()

tb_age_avg_df.head(10)

TOT_OFFENSES
COUNTY_NAME AGE_GROUP              
Anderson    10-19             17.50
            20-29             38.25
            30-39             30.00
            40-49             12.25
            50-59              9.75
            60-69              4.25
            70-79              4.00
Andrews     10-19             71.50
            20-29             72.00
            30-39             59.00

### Export additional csv's to be used in Tableau Dashboard visualizations

In [29]:
tb_ar_avg_df.to_csv('tb_tx_arrests_county.csv')
tb_age_avg_df.to_csv('tb_tx_arrests_county_by_age.csv')